#**Import libraries**

In [ ]:
#importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

In [ ]:
import keras # main keras package
from keras.models import Sequential # sequential model
from keras.layers import Dropout, Flatten, AveragePooling2D # layers with layers operations
from keras.layers import Dense,Conv2D  # layers types
from keras.layers import BatchNormalization

print("Keras version:", keras.__version__)

Keras version: 2.8.0


In [ ]:
import numpy as np 
import pandas as pd
import os, time
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import applications
from sklearn.model_selection import train_test_split
import tensorflow.keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D , Embedding , LSTM, GRU, Bidirectional,Dropout, SpatialDropout2D
from tensorflow.keras.layers import Dense, Dropout, Flatten , AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf 
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras import models
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import callbacks
from tensorflow.keras.layers import LeakyReLU
import os
import warnings
warnings.filterwarnings('ignore')


# **Load Text**

The first step is to load the text into memory.

We can develop a small function to load the entire text file into memory and return it. The function is called load_doc() and is listed below. Given a filename, it returns a sequence of loaded text.

In [ ]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

Using this function, we can load the cleaner version of the document in the file ‘republic_clean.txt‘ as follows:

In [ ]:
# load document
in_filename = 'republic_clean.txt'
doc = load_doc(in_filename)
print(doc[:100])

BOOK I.

I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up


Running this snippet loads the document and prints the first 100 characters as a sanity check.



# **Clean Text**

We need to transform the raw text into a sequence of tokens or words that we can use as a source to train the model.

Based on reviewing the raw text (above), below are some specific operations we will perform to clean the text. You may want to explore more cleaning operations yourself as an extension.
*  Replace ‘--‘ with a white space so we can split words better.
  
* Split words based on white space.
* Remove all punctuation from words to reduce the vocabulary size .
* Remove all words that are not alphabetic to remove standalone punctuation tokens.
* Normalize all words to lowercase to reduce the vocabulary size.

Vocabulary size is a big deal with language modeling. A smaller vocabulary results in a smaller model that trains faster.

We can implement each of these cleaning operations in this order in a function. Below is the function clean_doc() that takes a loaded document as an argument and returns an array of clean tokens.

In [ ]:
import string
# turn a doc into clean tokens
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

We can run this cleaning function on our loaded document and print out some of the tokens and statistics as a sanity check.

In [ ]:
# clean document
tokens = clean_doc(doc)
print(tokens[:100])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['book', 'i', 'i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ariston', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'bendis', 'the', 'thracian', 'artemis', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'festival', 'which', 'was', 'a', 'new', 'thing', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'inhabitants', 'but', 'that', 'of', 'the', 'thracians', 'was', 'equally', 'if', 'not', 'more', 'beautiful', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'spectacle', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'city', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'cephalus', 'chanced', 'to', 'catch', 'sight']
Total Tokens: 118684
Unique Tokens: 7409


First, we can see a  list of tokens that look cleaner than the raw text. We could remove the ‘Book I‘ chapter markers and more, but this is a good start.

# **Save Clean Text**

We can organize the long list of tokens into sequences of 50 input words and 1 output word.

That is, sequences of 51 words.

We can do this by iterating over the list of tokens from token 51 onwards and taking the prior 50 tokens as a sequence, then repeating this process to the end of the list of tokens.

We will transform the tokens into space-separated strings for later storage in a file.

The code to split the list of clean tokens into sequences with a length of 51 tokens is listed below.

In [ ]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 118633


In [ ]:
len(sequences)

118633

Running this piece creates a long list of lines.

Printing statistics on the list, we can see that we will have exactly 118,633 training patterns to fit our model.

Next, we can save the sequences to a new file for later loading.

We can define a new function for saving lines of text to a file. This new function is called save_doc() and is listed below. It takes as input a list of lines and a filename. The lines are written, one per line, in ASCII format.

In [ ]:
# save tokens to file, one dialog per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

We can call this function and save our training sequences to the file ‘republic_sequences.txt‘.



In [ ]:
# save sequences to file
out_filename = 'republic_sequences.txt'
save_doc(sequences, out_filename)

# **Train Language Model**

We can now train a statistical language model from the prepared data.

The model we will train is a neural language model. It has a few unique characteristics:
* It uses a distributed representation for words so that different words with similar meanings will have a similar representation.
* It learns the representation at the same time as learning the model.
* It learns to predict the probability for the next word using the context of the last 100 words.

Specifically, we will use an Embedding Layer to learn the representation of words, and a Long Short-Term Memory (LSTM) recurrent neural network to learn to predict words based on their context.

Let’s start by loading our training data.

**Load Sequences**


We can load our training data using the load_doc() function we developed in the previous section.

Once loaded, we can split the data into separate training sequences by splitting based on new lines.

The snippet below will load the ‘republic_sequences.txt‘ data file from the current working directory.

In [ ]:
# # load doc into memory
# def load_doc(filename):
# 	# open the file as read only
# 	file = open(filename, 'r')
# 	# read all text
# 	text = file.read()
# 	# close the file
# 	file.close()
# 	return text
 
# load document
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

Next, we can encode the training data.



**Encode Sequences**

The word embedding layer expects input sequences to be comprised of integers.

We can map each word in our vocabulary to a unique integer and encode our input sequences. Later, when we make predictions, we can convert the prediction to numbers and look up their associated words in the same mapping.

To do this encoding, we will use the Tokenizer class in the Keras API.

First, the Tokenizer must be trained on the entire training dataset, which means it finds all of the unique words in the data and assigns each a unique integer.

We can then use the fit Tokenizer to encode all of the training sequences, converting each sequence from a list of words to a list of integers.

In [ ]:
from keras.preprocessing.text import Tokenizer
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [ ]:
#display first line after encoding
print(sequences[2])

[11, 1045, 329, 7409, 4, 1, 2873, 35, 213, 1, 261, 3, 2251, 9, 11, 179, 817, 123, 92, 2872, 4, 1, 2249, 7408, 1, 7407, 7406, 2, 75, 120, 11, 1266, 4, 110, 6, 30, 168, 16, 49, 7405, 1, 1609, 13, 57, 8, 549, 151, 11, 57, 1147, 35]


We can access the mapping of words to integers as a dictionary attribute called word_index on the Tokenizer object.

We need to know the size of the vocabulary for defining the embedding layer later. We can determine the vocabulary by calculating the size of the mapping dictionary.

Words are assigned values from 1 to the total number of words (e.g. 7,409). The Embedding layer needs to allocate a vector representation for each word in this vocabulary from index 1 to the largest index and because indexing of arrays is zero-offset, the index of the word at the end of the vocabulary will be 7,409; that means the array must be 7,409 + 1 in length.

Therefore, when specifying the vocabulary size to the Embedding layer, we specify it as 1 larger than the actual vocabulary.

In [ ]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
vocab_size 

7410

# **Sequence Inputs and Output**

In [ ]:
# separate into input and output
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1] #50

#**Models**

## **Fit LSTM Model**

We can now define and fit our language model on the training data.

The learned embedding needs to know the size of the vocabulary and the length of input sequences as previously discussed. It also has a parameter to specify how many dimensions will be used to represent each word. That is, the size of the embedding vector space.


We will use a two LSTM hidden layers with 100 memory cells each. More memory cells and a deeper network may achieve better results.

A dense fully connected layer with 100 neurons connects to the LSTM hidden layers to interpret the features extracted from the sequence. The output layer predicts the next word as a single vector the size of the vocabulary with a probability for each word in the vocabulary. A softmax activation function is used to ensure the outputs have the characteristics of normalized probabilities

In [ ]:
# define model
LSTM_model = Sequential()
LSTM_model.add(Embedding(vocab_size, 50, input_length=seq_length))
LSTM_model.add(LSTM(100, return_sequences=True))
LSTM_model.add(LSTM(100))
LSTM_model.add(Dense(100, activation='relu'))
LSTM_model.add(Dense(vocab_size, activation='softmax'))


A summary of the defined network is printed as a sanity check to ensure we have constructed what we intended.

In [ ]:
print(LSTM_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            370500    
                                                                 
 lstm (LSTM)                 (None, 50, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 7410)              748410    
                                                                 
Total params: 1,269,810
Trainable params: 1,269,810
Non-trainable params: 0
_________________________________________________________________
None


Next, the model is compiled specifying the categorical cross entropy loss needed to fit the model. Technically, the model is learning a multi-class classification and this is the suitable loss function for this type of problem. The efficient Adam implementation to mini-batch gradient descent is used and accuracy is evaluated of the model.

Finally, the model is fit on the data for 100 training epochs with a modest batch size of 128 to speed things up.



In [ ]:
# compile model
LSTM_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
LSTM_model.fit(X, y, batch_size=300, epochs=100)

Epoch 1/100
396/396 [==============================] - 13s 19ms/step - loss: 6.3140 - accuracy: 0.0604
Epoch 2/100
396/396 [==============================] - 7s 18ms/step - loss: 5.8767 - accuracy: 0.0908
Epoch 3/100
396/396 [==============================] - 7s 18ms/step - loss: 5.6503 - accuracy: 0.1077
Epoch 4/100
396/396 [==============================] - 7s 18ms/step - loss: 5.4634 - accuracy: 0.1274
Epoch 5/100
396/396 [==============================] - 7s 18ms/step - loss: 5.3315 - accuracy: 0.1378
Epoch 6/100
396/396 [==============================] - 7s 19ms/step - loss: 5.2355 - accuracy: 0.1464
Epoch 7/100
396/396 [==============================] - 8s 20ms/step - loss: 5.1574 - accuracy: 0.1534
Epoch 8/100
396/396 [==============================] - 7s 19ms/step - loss: 5.0913 - accuracy: 0.1583
Epoch 9/100
396/396 [==============================] - 7s 19ms/step - loss: 5.0342 - accuracy: 0.1621
Epoch 10/100
396/396 [==============================] - 7s 19ms/step - loss: 4.98

### **Save Model**

At the end of the run, the trained model is saved to file.

Here, we use the Keras model API to save the model to the file ‘model.h5‘ in the current working directory.

Later, when we load the model to make predictions, we will also need the mapping of words to integers. This is in the Tokenizer object, and we can save that too using Pickle.

In [ ]:
from pickle import dump
# save the model to file
LSTM_model.save('LSTM_model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [ ]:
#clear session
tf.keras.backend.clear_session()

## **Fit GRU Model**

We can now define and fit our language model on the training data.

The learned embedding needs to know the size of the vocabulary and the length of input sequences as previously discussed. It also has a parameter to specify how many dimensions will be used to represent each word. That is, the size of the embedding vector space.


We will use a two GRU hidden layers with 100 memory cells each. More memory cells and a deeper network may achieve better results.

A dense fully connected layer with 100 neurons connects to the GRU hidden layers to interpret the features extracted from the sequence. The output layer predicts the next word as a single vector the size of the vocabulary with a probability for each word in the vocabulary. A softmax activation function is used to ensure the outputs have the characteristics of normalized probabilities

In [ ]:
# define model
GRU_model = Sequential()
GRU_model.add(Embedding(vocab_size, 50, input_length=seq_length))
GRU_model.add(GRU(100, return_sequences=True))
GRU_model.add(GRU(100))
GRU_model.add(Dense(100, activation='relu'))
GRU_model.add(Dense(vocab_size, activation='softmax'))


A summary of the defined network is printed as a sanity check to ensure we have constructed what we intended.

In [ ]:
print(GRU_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            370500    
                                                                 
 gru (GRU)                   (None, 50, 100)           45600     
                                                                 
 gru_1 (GRU)                 (None, 100)               60600     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 7410)              748410    
                                                                 
Total params: 1,235,210
Trainable params: 1,235,210
Non-trainable params: 0
_________________________________________________________________
None


Next, the model is compiled specifying the categorical cross entropy loss needed to fit the model. Technically, the model is learning a multi-class classification and this is the suitable loss function for this type of problem. The efficient Adam implementation to mini-batch gradient descent is used and accuracy is evaluated of the model.

Finally, the model is fit on the data for 100 training epochs with a modest batch size of 300 to speed things up.



In [ ]:
# compile model
GRU_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
GRU_model.fit(X, y, batch_size=300, epochs=100)

Epoch 1/100
396/396 [==============================] - 12s 18ms/step - loss: 6.3576 - accuracy: 0.0576
Epoch 2/100
396/396 [==============================] - 7s 18ms/step - loss: 6.1412 - accuracy: 0.0597
Epoch 3/100
396/396 [==============================] - 8s 19ms/step - loss: 6.1373 - accuracy: 0.0595
Epoch 4/100
396/396 [==============================] - 7s 18ms/step - loss: 6.1352 - accuracy: 0.0592
Epoch 5/100
396/396 [==============================] - 7s 18ms/step - loss: 6.1207 - accuracy: 0.0600
Epoch 6/100
396/396 [==============================] - 7s 18ms/step - loss: 5.9029 - accuracy: 0.0862
Epoch 7/100
396/396 [==============================] - 7s 18ms/step - loss: 5.6351 - accuracy: 0.1153
Epoch 8/100
396/396 [==============================] - 7s 18ms/step - loss: 5.4319 - accuracy: 0.1382
Epoch 9/100
396/396 [==============================] - 7s 18ms/step - loss: 5.2754 - accuracy: 0.1537
Epoch 10/100
396/396 [==============================] - 7s 18ms/step - loss: 5.13

### **Save Model**

At the end of the run, the trained model is saved to file.

Here, we use the Keras model API to save the model to the file ‘model.h5‘ in the current working directory.

Later, when we load the model to make predictions, we will also need the mapping of words to integers. This is in the Tokenizer object, and we can save that too using Pickle.

In [ ]:
from pickle import dump
# save the model to file
GRU_model.save('GRU_model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

## **Fit BI-Directional LSTM Model**

We can now define and fit our language model on the training data.

The learned embedding needs to know the size of the vocabulary and the length of input sequences as previously discussed. It also has a parameter to specify how many dimensions will be used to represent each word. That is, the size of the embedding vector space.


We will use a two LSTM and BI-Directional LSTM  hidden layers  with 100 memory cells each. More memory cells and a deeper network may achieve better results.

A dense fully connected layer with 300 neurons connects to the BI-Directional LSTM hidden layers to interpret the features extracted from the sequence. The output layer predicts the next word as a single vector the size of the vocabulary with a probability for each word in the vocabulary. A softmax activation function is used to ensure the outputs have the characteristics of normalized probabilities

In [ ]:
# define model
BI_LSTM_model = Sequential()
BI_LSTM_model.add(Embedding(vocab_size, 50, input_length=seq_length))
BI_LSTM_model.add(LSTM(100, return_sequences=True))
BI_LSTM_model.add( Bidirectional (LSTM(100)))
BI_LSTM_model.add(Dense(300, activation='relu'))
BI_LSTM_model.add(Dense(vocab_size, activation='softmax'))


A summary of the defined network is printed as a sanity check to ensure we have constructed what we intended.

In [ ]:
print(BI_LSTM_model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            370500    
                                                                 
 lstm (LSTM)                 (None, 50, 100)           60400     
                                                                 
 bidirectional (Bidirectiona  (None, 200)              160800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 300)               60300     
                                                                 
 dense_1 (Dense)             (None, 7410)              2230410   
                                                                 
Total params: 2,882,410
Trainable params: 2,882,410
Non-trainable params: 0
______________________________________________

Next, the model is compiled specifying the categorical cross entropy loss needed to fit the model. Technically, the model is learning a multi-class classification and this is the suitable loss function for this type of problem. The efficient Adam implementation to mini-batch gradient descent is used and accuracy is evaluated of the model.

Finally, the model is fit on the data for 100 training epochs with a modest batch size of 300 to speed things up.
.

In [ ]:
# compile model
BI_LSTM_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
BI_LSTM_model.fit(X, y, batch_size=300, epochs=100)

Epoch 1/100
396/396 [==============================] - 17s 24ms/step - loss: 6.2740 - accuracy: 0.0618
Epoch 2/100
396/396 [==============================] - 10s 24ms/step - loss: 5.8455 - accuracy: 0.0927
Epoch 3/100
396/396 [==============================] - 10s 24ms/step - loss: 5.6362 - accuracy: 0.1106
Epoch 4/100
396/396 [==============================] - 10s 24ms/step - loss: 5.4559 - accuracy: 0.1314
Epoch 5/100
396/396 [==============================] - 10s 24ms/step - loss: 5.3257 - accuracy: 0.1424
Epoch 6/100
396/396 [==============================] - 10s 24ms/step - loss: 5.2277 - accuracy: 0.1490
Epoch 7/100
396/396 [==============================] - 10s 25ms/step - loss: 5.1543 - accuracy: 0.1530
Epoch 8/100
396/396 [==============================] - 10s 25ms/step - loss: 5.0728 - accuracy: 0.1589
Epoch 9/100
396/396 [==============================] - 10s 25ms/step - loss: 5.0036 - accuracy: 0.1627
Epoch 10/100
396/396 [==============================] - 10s 25ms/step - l

After we run our BI-Directional LSTM model, we observed that an accuracy increases for each epoch so we can increase the number of epochs to make good prediction for the next word in the sequence, which is not bad. We are not aiming for 100% accuracy

### **Save Model**

At the end of the run, the trained model is saved to file.

Here, we use the Keras model API to save the model to the file ‘model.h5‘ in the current working directory.

Later, when we load the model to make predictions, we will also need the mapping of words to integers. This is in the Tokenizer object, and we can save that too using Pickle.

In [ ]:
from pickle import dump
# save the model to file
BI_LSTM_model.save('BI_LSTM_model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [ ]:
#clear session
tf.keras.backend.clear_session()

## **Fit BI-Directional GRU Model**

We can now define and fit our language model on the training data.

The learned embedding needs to know the size of the vocabulary and the length of input sequences as previously discussed. It also has a parameter to specify how many dimensions will be used to represent each word. That is, the size of the embedding vector space.


We will use a two gru and Bidirectional hidden layers with 128 and 64 memory cells each. More memory cells and a deeper network may achieve better results.

A dense fully connected layer with 300 neurons connects to the bidirectional hidden layers to interpret the features extracted from the sequence. The output layer predicts the next word as a single vector the size of the vocabulary with a probability for each word in the vocabulary. A softmax activation function is used to ensure the outputs have the characteristics of normalized probabilities

In [ ]:
# define model
BI_GRU_model = Sequential( )
BI_GRU_model.add (Embedding(vocab_size , 50 , input_length = seq_length ))
BI_GRU_model.add (GRU(128 , return_sequences = True )) 
BI_GRU_model.add (Bidirectional (GRU(64))) 
BI_GRU_model.add (Dense (300 , activation = 'relu'))
BI_GRU_model.add (Dense(vocab_size , activation = 'softmax')) 

A summary of the defined network is printed as a sanity check to ensure we have constructed what we intended.

In [ ]:
print(BI_GRU_model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 50)            370500    
                                                                 
 gru_1 (GRU)                 (None, 50, 128)           69120     
                                                                 
 bidirectional (Bidirectiona  (None, 128)              74496     
 l)                                                              
                                                                 
 dense (Dense)               (None, 300)               38700     
                                                                 
 dense_1 (Dense)             (None, 7410)              2230410   
                                                                 
Total params: 2,783,226
Trainable params: 2,783,226
Non-trainable params: 0
____________________________________________

Next, the model is compiled specifying the categorical cross entropy loss needed to fit the model. Technically, the model is learning a multi-class classification and this is the suitable loss function for this type of problem. The efficient Adam implementation to mini-batch gradient descent is used and accuracy is evaluated of the model.

Finally, the model is fit on the data for 100 training epochs with a modest batch size of 128 to speed things up.
.

In [ ]:
# compile model
BI_GRU_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
BI_GRU_model.fit(X, y, batch_size=300 , epochs=100)

Epoch 1/100
396/396 [==============================] - 17s 21ms/step - loss: 6.3507 - accuracy: 0.0580
Epoch 2/100
396/396 [==============================] - 8s 21ms/step - loss: 5.9909 - accuracy: 0.0743
Epoch 3/100
396/396 [==============================] - 8s 21ms/step - loss: 5.5265 - accuracy: 0.1241
Epoch 4/100
396/396 [==============================] - 8s 21ms/step - loss: 5.1849 - accuracy: 0.1505
Epoch 5/100
396/396 [==============================] - 8s 21ms/step - loss: 4.9524 - accuracy: 0.1638
Epoch 6/100
396/396 [==============================] - 8s 21ms/step - loss: 4.7605 - accuracy: 0.1760
Epoch 7/100
396/396 [==============================] - 8s 21ms/step - loss: 4.5845 - accuracy: 0.1856
Epoch 8/100
396/396 [==============================] - 8s 21ms/step - loss: 4.4081 - accuracy: 0.1938
Epoch 9/100
396/396 [==============================] - 8s 21ms/step - loss: 4.2269 - accuracy: 0.2024
Epoch 10/100
396/396 [==============================] - 8s 21ms/step - loss: 4.03

After we run our BI directional GRU model, we observed that an accuracy increases for each epoch so we can increase the number of epochs to make good prediction for the next word in the sequence

### **Save Model**

At the end of the run, the trained model is saved to file.

Here, we use the Keras model API to save the model to the file ‘model.h5‘ in the current working directory.

Later, when we load the model to make predictions, we will also need the mapping of words to integers. This is in the Tokenizer object, and we can save that too using Pickle.

In [ ]:
from pickle import dump
# save the model to file
BI_GRU_model.save('BI_GRU_model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

# **Use Language Model**
Now that we have a trained language model, we can use it.

In this case, we can use it to generate new sequences of text that have the same statistical properties as the source text.


This is not practical, at least not for this example, but it gives a concrete example of what the language model has learned.


We will start by loading the training sequences again.

# **Generate Text**
The first step in generating text is preparing a seed input.

We will select a random line of text from the input text for this purpose.



We can create a function called generate_seq() that takes as input the model, the tokenizer, input sequence length, the seed text, and the number of words to generate. It then returns a sequence of words generated by the model.

In [ ]:
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		# yhat = model.predict_classes(encoded, verbose=0)
		yhat = np.argmax(model.predict(np.array(encoded)), axis=1)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)


In [ ]:
# load cleaned text sequences
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

seq_length = len(lines[0].split()) - 1
 
# load the model
LSTM_model  = load_model('LSTM_model.h5')
GRU_model   = load_model('GRU_model.h5')
BI_LSTM_modeL = load_model('BI_LSTM_modeL.h5')
BI_GRU_model = load_model('BI_GRU_model.h5')
 
# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))
 
# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

slavery more than death undoubtedly also we shall have to reject all the terrible and appalling names which describe the world below cocytus and styx ghosts under the earth and sapless shades and any similar words of which the very mention causes a shudder to pass through the inmost soul of



We are now ready to generate a sequence of new words given some seed text.



In [ ]:
# generate new text
generated = generate_seq(LSTM_model, tokenizer, seq_length, seed_text, 50)
print(generated)

to be attributed to their fine wits time to the islands of the blest and dwell and of the whole state and the meaner perfect is the sun then we shall be satisfied with that the eye is not the penalties and retributions and they will erase and another man


In [ ]:
# generate new text
generated = generate_seq(GRU_model, tokenizer, seq_length, seed_text, 50)
print(generated)

not the several persons who are heard in the institution of gymnastic also rather than themselves and that he dragged even to range in the view or themselves and to be compared with the world only to his custom and not be punished but when he has a lover of


In [ ]:
# generate new text
generated = generate_seq(BI_LSTM_modeL, tokenizer, seq_length, seed_text, 50)
print(generated)

extremely called the authors of the good vary but who has him persuade before them therefore they are seeking not useless with tools or the injustice of plunder yes he said i expect that those and educators which are the distance and not therefore you have the most aggravated city


In [ ]:
# generate new text
generated = generate_seq(BI_GRU_model, tokenizer, seq_length, seed_text, 50)
print(generated)

not the happier which the embroiderer what you say i cannot examine giving those be or as our state which is the question which i will to reflect what do you say of the situation have the same object of repose you mean to hear the informer of democracy we


**Note**: The results may vary given the stochastic nature of the algorithm or evaluation procedure, or differences in numerical precision.

we can see that the text seems reasonable. In fact, the addition of concatenation would help in interpreting the seed and the generated text. Nevertheless, the generated text gets the right kind of words in the right kind of order.
